**注意事項**

このノートブックは、GPU:「T4」に対応させたものです。
「L4」版のノートブックとはモデル等が異なるため、生成される内容が異なることが考えられます。

生成される内容と、ノートブックに記載されている説明が一致しない場合があることをご了承ください。

生成内容とノートブックの説明をよく見比べ、適宜読み替えながら演習を進めてみてください。

---

# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察します。これにより、特別な学習なしでのモデルの限界を確認します。

2. **文字起こしデータの活用**  
   講義の文字起こしデータを導入し、モデルが講義内容を参照した回答を生成する傾向を観察します。ただし、Retrieval（情報検索）精度の限界から結果は不安定になる可能性があります。

3. **チャンク化の導入**  
   文字起こしデータをチャンク（小単位）に分割し、より安定して関連コンテンツを取得できるようにします。この段階では文脈理解にまだ課題があることを確認します。

4. **Rerankの適用**  
   検索結果のランク付けを導入し、より的確で安定した回答を目指します。

5. **応用改善手法**  
   文字起こしの品質向上のための編集技術や、メタデータの活用による性能向上手法を探ります。

## 扱う質問

「Inference Time Scaling（推論時スケーリング）」に関する質問を取り扱います。これは以下の背景を持つトピックです。

- 2024年8月発表の論文「Scaling LLM Test-Time Compute Optimally can be More Effective than Scaling Model Parameters」で提唱された概念
- OpenAIのGPT-o1（2024年9月リリース）で実用化され、注目を集めた比較的新しいアプローチ
- 2024年度LLM講座の第4回講義でも取り上げられた重要テーマ

## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用します。このモデルは、リリース時期の関係上、以下の特徴を持ちます。

- 「Inference Time Scaling」の概念が広まる前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

### 演習環境の準備

In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 30.9 MB/s eta 0:00:0000:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 25.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 8.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.0 MB/s eta 0:00:0000:0100:01


In [1]:
!pip show transformers
# バージョンが古い場合は
#pip install --upgrade transformers

Name: transformers
Version: 4.51.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [2]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

fatal: destination path 'lecture-ai-engineering' already exists and is not an empty directory.


In [2]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import random
random.seed(0)

In [5]:
import torch
# モデル(Qwen3-8B)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "Qwen/Qwen3-8B-FP8"
# 2. トークナイザーの読み込み (モデル名に合わせて自動で変更)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True # Qwenモデルはカスタムコードを含む場合があるのでTrueを推奨
)

# 3. BitsAndBytesConfig (量子化設定)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_4bit_compute_dtype: Qwenモデルは通常bfloat16で学習されています。
    # GPUがbfloat16に対応している場合 (NVIDIA Ampere世代以降など) は torch.bfloat16 を推奨。
    # 対応していない場合は torch.float16 を使用します。
    # モデルカードのQuickstartでは torch_dtype="auto" となっていますが、
    # 4bit量子化と組み合わせる際は明示的に指定するのが良いでしょう。
    bnb_4bit_compute_dtype=torch.bfloat16, # または torch.float16
    bnb_4bit_quant_type="nf4", # 一般的に高性能なので維持
    bnb_4bit_use_double_quant=False, # まずはFalseで。VRAMが非常に厳しい場合にTrueを検討
)
# 4. モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", # 維持
    quantization_config=bnb_config, # 4bit量子化設定を適用
    # torch_dtype: 量子化時のベースとなるデータ型。bnb_4bit_compute_dtype と合わせるか、
    # モデルがネイティブにサポートする型 (Qwenの場合はbfloat16が適していることが多い) を指定。
    torch_dtype=torch.bfloat16, # または torch.float16
    trust_remote_code=True, # QwenモデルではTrueが必要なことが多い
    # attn_implementation: FlashAttention2が利用可能ならパフォーマンス向上に寄与する可能性
    # attn_implementation="flash_attention_2", # 必要ならインストールして試す
    # rope_scaling: 長いコンテキスト (YaRN) を使う場合に設定。後述。
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

In [15]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available. Running on CPU.")

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
Number of GPUs: 1
Current CUDA device: 0
Device name: NVIDIA GeForce RTX 4080


# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [12]:
def generate_output_qwen3(query, model, tokenizer, enable_thinking=True, max_new_tokens=512):
    """
    Qwen3モデル用に調整されたテキスト生成関数。

    Args:
        query (str): モデルへの質問文。
        model: ロード済みのQwen3モデル。
        tokenizer: Qwen3モデルに対応するトークナイザー。
        enable_thinking (bool): Qwen3のThinking Modeを有効にするか。Trueで有効。
        max_new_tokens (int): 生成する最大トークン数。

    Returns:
        str: モデルからの応答テキスト（Thinking Modeの場合は最終回答部分）。
    """
    messages = [
        {"role": "user", "content": query},
    ]

    # 1. チャットテンプレートの適用 (enable_thinking を追加)
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        enable_thinking=enable_thinking # Qwen3特有の引数
    ).to(model.device)

    # 2. 終了トークンの設定
    # Qwen3では、tokenizer.eos_token_id が主要な終了トークンです。
    # <|eot_id|> は他のモデル(Llama 3など)の特殊トークンであり、Qwen3では通常不要か、
    # 存在しない場合はエラーになる可能性があります。
    # Qwen3の特殊トークンには <|im_end|> や <|endoftext|> がありますが、
    # apply_chat_templateがこれらを適切に扱うため、eos_token_idのみで十分なことが多いです。
    eos_token_id_to_use = tokenizer.eos_token_id
    # 必要に応じて他の終了候補もリストに追加できますが、まずはeos_token_idだけで試すのが無難です。
    # terminators = [tokenizer.eos_token_id]
    # もしtokenizer.eos_token_idがNoneの場合は代替を探す
    if eos_token_id_to_use is None and hasattr(tokenizer, 'eod_id'): # Qwen特有の属性
        eos_token_id_to_use = tokenizer.eod_id


    # 3. サンプリングパラメータの設定
    generation_kwargs = {
        "max_new_tokens": max_new_tokens,
        "eos_token_id": eos_token_id_to_use,
        "pad_token_id": eos_token_id_to_use, # warning: `pad_token_id` is not set... を避ける
    }

    if enable_thinking:
        # Thinking Mode (enable_thinking=True) の場合の推奨値
        # モデルカードではgreedy decodingは非推奨なので do_sample=True
        generation_kwargs.update({
            "do_sample": True,
            "temperature": 0.6,
            "top_p": 0.95,
            "top_k": 20,
            # "min_p": 0, # モデルカードに記載あり。必要に応じて追加
        })
    else:
        # Non-Thinking Mode (enable_thinking=False) の場合の推奨値
        generation_kwargs.update({
            "do_sample": True,
            "temperature": 0.7,
            "top_p": 0.8,
            "top_k": 20,
            # "min_p": 0, # モデルカードに記載あり。必要に応じて追加
        })
    
    # もし strictly greedy な出力を試したい場合 (モデルカードでは非推奨だが比較用として)
    # generation_kwargs = {
    #     "max_new_tokens": max_new_tokens,
    #     "eos_token_id": eos_token_id_to_use,
    #     "pad_token_id": eos_token_id_to_use,
    #     "do_sample": False, # greedy search
    # }

    # 4. テキスト生成
    outputs = model.generate(
        input_ids,
        **generation_kwargs
    )

    # 5. 応答部分の抽出とデコード
    response_ids = outputs[0][input_ids.shape[-1]:]

    if enable_thinking:
        # Thinking Mode の場合、<think>...</think> タグの後に最終回答がある
        full_response_decoded = tokenizer.decode(response_ids, skip_special_tokens=False) # 特殊トークンを含めてデコード
        
        think_end_tag = "</think>"
        final_answer = ""
        
        think_end_idx = full_response_decoded.rfind(think_end_tag)
        if think_end_idx != -1:
            raw_final_answer = full_response_decoded[think_end_idx + len(think_end_tag):]
            # 特殊トークンを除去して最終回答をクリーンにする
            final_answer = tokenizer.decode(tokenizer.encode(raw_final_answer, add_special_tokens=False), skip_special_tokens=True).strip()
        else:
            # <think> タグが見つからない場合は、全体をデコード (特殊トークン除去)
            final_answer = tokenizer.decode(response_ids, skip_special_tokens=True).strip()
        
        return final_answer
    else:
        # Non-Thinking Mode の場合は、単純にデコード
        return tokenizer.decode(response_ids, skip_special_tokens=True).strip()

神戸情報大学院大学
〒650-0001 兵庫県神戸市中央区加納町２丁目１−１５
https://www.kic.ac.jp/
現学長：炭谷　俊樹

In [ ]:
question =  "LLMにおけるInference Time Scalingとは？"
response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
print(f"Thinking ON: {response_on}")
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

In [49]:
question =  "神戸情報大学院大学はどこにある？"
response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
print(f"Thinking ON: {response_on}")
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking ON: <think>
Okay, the user is asking where Kobe Institute of Computing is located. First, I need to confirm the correct name. The user wrote "神戸情報大学院大学" which translates to Kobe Institute of Computing. Wait, but I remember that the official English name might be different. Let me check.

Wait, actually, the university's official English name is Kobe Institute of Computing, but sometimes it's also referred to as Kobe University of Information and Computing. Wait, no, maybe I'm mixing it up. Let me verify.

Looking up, the correct English name is Kobe Institute of Computing. The Japanese name is 神戸情報大学院大学. So the user is correct in their translation. Now, the location. The university is located in Kobe, Japan. But which district? I think it's in the city of Kobe, specifically in the Higashi-hiroshima area? Or maybe in the Sannomiya district? Wait, I need to be precise.

Wait, Kobe Institute of Computing is actually part of Kobe University. Wait, no, that's not right. Wait, no, K

In [19]:
question =  "神戸情報大学院大学の現学長は？"
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking OFF: 2024年現在、神戸情報大学院大学の現学長は **小林 晋一（こばやし しんいち）氏** です。

### 学長の略歴（一部）：
- 1962年生まれ
- 京都大学大学院理学研究科博士課程修了（物理学科）
- 京都大学大学院理学研究科教授、学術研究振興事業団（ARO）所属研究員、米国カリフォルニア大学バークレー校客員研究員などを経て、神戸情報大学院大学学長に就任

### 研究分野：
- 理論物理、統計力学、複雑系物理、情報物理など

### 略歴に関する注意点：
- 神戸情報大学院大学は、2020年4月に「神戸大学大学院情報学研究科」が設置され、それ以前は「神戸情報大学院大学」として運営されていました。
- 学長の人事は年次で変更されることがあるため、最新情報は公式ウェブサイトで確認することをおすすめします。

---

### 公式ウェブサイト：
- [神戸情報大学院大学 公式サイト](https://www.kobe-u.ac.jp/)

必要であれば、学長の主な活動や研究テーマについても詳しくお伝えできます。


In [14]:
question = "神戸情報大学院大学学則、第一章第一条は？"

# Thinking Mode を有効にして生成
response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
print(f"Thinking ON: {response_on}")

# Thinking Mode を無効にして生成
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking ON: 神戸情報大学院大学の学則（学則）における**第一章第一条**は、大学の基本的な設立目的や名称、設置場所などの一般的な規定を記載しています。ただし、具体的な条文は大学の公式資料に記載されており、外部からは直接取得できない場合があります。以下に
Thinking OFF: 神戸情報大学院大学の学則（学則は大学が学生に対して定める規則）の第一章第一条は、一般的に大学の設立目的や基本理念を示す条文として、大学の設立にかかわる内容が記載されています。

ただし、具体的な内容は、神戸情報大学院大学の公式サイトや学則の原文を参照する必要があります。

以下は、一般的な大学の学則第一章第一条の内容の例です：

> **第一章 総則**  
> **第一条（設立目的）**  
> この大学は、社会のニーズに応じた高度な専門的知識と技術を備えた人材の育成を目的として、教育・研究を通じて社会の発展に貢献する。

ただし、**神戸情報大学院大学**の学則第一章第一条の正確な内容は、以下の方法で確認できます：

1. **公式ウェブサイト**  
   神戸情報大学院大学の公式ウェブサイト（https://www.kobe-u.ac.jp/）から「学則」や「大学規則」を検索してください。

2. **大学のパンフレットや資料**  
   新入生向けパンフレットや大学の資料集に掲載されている場合があります。

3. **大学の事務局や学務課**  
   学則の原文を直接確認したい場合は、大学の事務局や学務課に問い合わせてください。

---

もし、**神戸情報大学院大学**の学則第一章第一条の具体的な内容を知りたい場合、その原文を教えていただければ、それに基づいて解説も可能です。


In [21]:
question =  "神戸情報大学院大学学則において定められている修業年限の限度は？"
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking OFF: 神戸情報大学院大学（こうべじょうほうだいがくいんだいがく）の学則においては、修業年限の限度について明記されていますが、具体的な数字は大学の公式資料（学則）に記載されているため、正確な情報はその最新版を確認する必要があります。

一般的に、日本の大学においては、**4年制の学部**や**大学院**では、**修業年限の上限が定められており、通常は4年（学部）または2年（大学院）**とされています。ただし、大学によっては例外や特別な制度がある場合もあります。

### 神戸情報大学院大学のケースについて
神戸情報大学院大学は、**情報学部**と**大学院情報学研究科**を設置しており、学部と大学院の制度がそれぞれ異なるため、それぞれの修業年限が異なる可能性があります。

#### 1. 学部（情報学部）
- **修業年限の上限**：通常、**4年**（4年制）が一般的ですが、**6年以内**とされている場合があります。
- 学則には「修業年限の限度」として、**6年以内**と定められている場合があります。

#### 2. 大学院（情報学研究科）
- **修業年限の上限**：通常、**2年**（修士課程）が一般的ですが、**4年以内**とされている場合があります。
- 学則には「修業年限の限度」として、**4年以内**と定められている場合があります。

---

### まとめ
- **学部（情報学部）**：**6年以内**（修業年限の限度）
- **大学院（情報学研究科）**：**4年以内**（修業年限の限度）

---

### おすすめの確認方法
- 神戸情報大学院大学の**公式ウェブサイト**（特に「学則」ページ）を確認してください。
- 学則のPDFファイルをダウンロードし、**「修業年限」**や**「修業年限の限度」**の項目を検索してください。
- 学務課や学則担当部署に直接問い合わせることも可能です。

必要であれば、学則のPDFファイルを提供するリンクや、具体的な条文を引用する


In [16]:
question =  "神戸情報大学院大学学則において定められている入学金はいくら？"
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking OFF: 神戸情報大学院大学の学則における**入学金**の金額は、**年次によって異なり**、また**学部・大学院の違い**によっても変化します。

### 以下は、一般的な入学金の目安（2024年度）です。ただし、最新の金額は必ず**大学公式ウェブサイト**または**入試案内**で確認してください。

---

## ✅ 神戸情報大学院大学の入学金（2024年度目安）

| 学部・大学院 | 入学金（予定） |
|--------------|----------------|
| **学部（4年制）** | 約 **200,000円** |
| **大学院（修士課程）** | 約 **200,000円** |
| **大学院（博士課程）** | 約 **200,000円** |

---

### 🔍 注意点

- 上記の金額は**予定**であり、**実際の金額は学則の改訂や入試制度の変更によって変更される可能性があります**。
- 入学金は**一度支払うと、卒業まで有効**となる場合があります。
- 一部の学部やコース（例：特定の専攻や国際プログラム）では、**追加の費用**がかかる場合があります。

---

### 📘 参考：公式サイト

神戸情報大学院大学の公式ウェブサイトから、**学則**や**入試案内**、**学費・入学金の詳細**を確認できます。

- [神戸情報大学院大学 公式サイト](https://www.kobe-u.ac.jp/)

---

### 📌 まとめ

- **神戸情報大学院大学の入学金（2024年度）は、学部・大学院問わず約20万円**。
- 最新の情報は**公式サイト**で確認してください。

ご不明な点があれば、大学の**入試課**や**総務課**に直接お問い合わせいただくのが確実です。


In [24]:
question =  "神戸情報大学院大学学則において定められている、入学時にかかる費用は総額で幾ら？"
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking OFF: 神戸情報大学院大学（こうべ情報大学院大学）の学則に定められている「入学時にかかる費用」の総額について、具体的な金額は年によって変動し、また学部やコースによっても異なります。そのため、正確な金額を示すには、以下の点を確認する必要があります。

---

### ✅ 1. 入学時にかかる費用の主な項目
入学時に支払う費用は、一般的に以下のものがあります：

1. **入学金**（入学者に対する一括金）
2. **授業料**（入学時または年間の授業料）
3. **教材費**（教材や実習費など）
4. **入試料**（入試に応募する際に支払う費用）
5. **登録料**（学籍登録にかかる費用）
6. **保険料**（学生向けの保険に加入する場合）

---

### ✅ 2. 神戸情報大学院大学の入学費用（2024年度予定）

以下の情報は、**2024年度の予定**（2023年時点での情報）に基づいています。正確な金額は、**大学の公式ウェブサイト**または**入試案内**で確認してください。

#### ✅ 入学金（入学者に対する一括金）
- **100,000円**（※学部・コースによって異なる場合あり）

#### ✅ 授業料（年間）
- **450,000円**（※学部・コースによって異なる場合あり）

#### ✅ 教材費（年間）
- **100,000円〜200,000円**（※コースによって異なる）

#### ✅ 入試料
- **10,000円〜20,000円**（※入試種別によって異なる）

#### ✅ 登録料
- **10,000円**（※学部・コースによって異なる）

#### ✅ 保険料
- **10,000円〜30,000円**（※保険内容によって異なる）

---

### ✅ 総額の目安（入学時）


In [25]:
question =  "神戸情報大学院大学はどんな学校？"
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

Thinking OFF: 神戸情報大学院大学（こうべじょうほうがくいんだいがく）は、日本の私立大学であり、主に情報学・コンピュータ関連の分野に特化した大学院（大学院大学）です。以下にその特徴や概要を紹介します。

---

## 🔹概要

- **所在地**：兵庫県神戸市中央区
- **設立**：2004年（平成16年）
- **種別**：私立大学院大学
- **学部・大学院の構成**：
  - 大学院（大学院大学）として設置されているため、学部（本科）は設置されていません。
  - 主に**修士課程**と**博士課程**が設置されています。

---

## 🔹教育内容

神戸情報大学院大学は、**情報学・コンピュータ科学・情報通信技術**を軸にした高度な専門知識を学ぶことを目的としています。特に、以下のような分野が特徴的です：

- **情報工学**
- **コンピュータ科学**
- **情報通信工学**
- **情報処理**
- **ネットワーク技術**
- **ソフトウェア工学**
- **AI・人工知能**
- **情報セキュリティ**
- **データベース・情報システム**

---

## 🔹特徴

### ✅ 専門性の高い教育
- 専門性の高い情報系分野に特化しており、実践的な研究や技術開発に重点を置いた教育を行っています。
- 多くの研究室が産業界と連携しており、実社会での応用が重視されています。

### ✅ 専門職大学院としての位置づけ
- 専門職大学院（専門職学位大学院）として設置されており、**修士号（専門職修士）**や**博士号（専門職博士）**を取得できる大学院です。
- 専門職学位制度は、実践的な知識とスキルを身につけた人材を育成することを目的としています。

### ✅ 就職支援
- 専門職大学院の特徴として、産業界との連携が


- 数値的な評価も見てみます。RagasにはAnswer Accuracyという評価指標があります。今回はこちらを参考に実装した評価関数を利用して測っていきます。

- 今回はgemmaでは性能が不安定だったので、OpenAIのgpt-4oで評価していきます。従って、scoreの実行はopenAI APIキーを所持している関心がある方のみで良いです。

In [9]:
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.3/661.3 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0


In [22]:
# @title 評価実装
# gold_answer = "「Inference Time Scaling」とは、推論時に計算量を増やしてモデルの性能を高める手法です。これはモデルのサイズを大きくする代わりに、難しい入力に対して多くの計算リソースを使うことで、より良い出力を得ようとするアプローチです。"

# from openai import OpenAI
# from google.colab import userdata
# client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

# def openai_generator(query):

#         messages = [
#                     {
#                         "role": "user",
#                         "content": query
#                     }
#                 ]

#         response = client.chat.completions.create(
#             model="gpt-4o-mini",
#             messages=messages
#         )
#         return response.choices[0].message.content

# def evaluate_answer_accuracy(query, response, reference):

#     template_accuracy1 = (
#           "Instruction: You are a world class state of the art assistant for rating "
#           "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
#           "Say 4, if User Answer is full contained and equivalent to Reference Answer"
#           "in all terms, topics, numbers, metrics, dates and units.\n"
#           "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
#           "in all terms, topics, numbers, metrics, dates and units.\n"
#           "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
#           "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
#           "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
#           "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
#           "### Question: {query}\n"
#           "### {answer0}: {sentence_inference}\n"
#           "### {answer1}: {sentence_true}\n"
#           "The rating is:\n"
#       )
#     template_accuracy2 = (
#           "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
#           "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
#           "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
#           "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
#           "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
#           "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
#           "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
#           "Question: {query}\n\n"
#           "{answer0}: {sentence_inference}\n\n"
#           "{answer1}: {sentence_true}\n\n"
#           "Rating: "
#       )

#     score1 = openai_generator(
#                 template_accuracy1.format(
#                       query=query,
#                       answer0="User Answer",
#                       answer1="Reference Answer",
#                       sentence_inference=response,
#                       sentence_true=reference,
#                     )
#                 )
#     try:
#       score1 = int(score1)
#     except:
#       print("Failed")
#       score1 = 0

#     score2 = openai_generator(
#                 template_accuracy2.format(
#                         query=query,
#                         answer0="Reference Answer",
#                         answer1="User Answer",
#                         sentence_inference=reference,
#                         sentence_true=response,
#                     )
#                   )

#     try:
#       score2 = int(score2)
#     except:
#       print("Failed")
#       score2 = 0


#     return (score1 + score2) / 2

In [23]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

## 結果 (ベースモデル)

「google/gemma-2-2b-jpn-it」は「Inference Time Scaling」について誤った知識を提示しました：
* モデルは従来の「推論時間の短縮」という文脈でInference Time Scalingを解釈しており、これはLLM分野における最新の「Inference Time Scaling」概念（推論時計算資源の最適配分）とは異なる説明になります。

---

# 2. 文字起こしデータの活用

## 2.1 講義内容をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: LLM講座第4講における講師の発言内容
* **目的**: モデルに「Inference Time Scaling」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 音声認識モデル(speech2text)で書き起こした生テキストをそのまま使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [26]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [40]:
!ls

AI-E-03  DL-B  lecture-ai-engineering  simplechat


In [41]:
with open("AI-E-03/RAG/kic-gakusoku.md", "r") as f:
  raw_writedown = f.read()

In [44]:
import re
# ドキュメントを用意する。
chapter_pattern = r"(^第\d+章\s*.*?$)"
# re.split() は区切り文字自体もリストに含めることがあるので、
# re.split() の挙動を利用して、区切り文字の直前で分割するようにする。
# (具体的には、lookahead assertion `(?=...)` を使うと便利)
# lookahead assertion を使うと、マッチした部分を消費せずに区切り位置とできる
split_pattern = r"(?=第\d+章)" # これで「# 第X章」の直前で分割

# re.split() を使って章ごとに分割
# ただし、最初の要素が空になる可能性があるので、フィルタリングする
# また、冒頭に章タイトル以外のテキストがある場合も考慮
# split_pattern で分割すると、各章のタイトルが各要素の先頭に来る
chapters_raw = re.split(split_pattern, raw_writedown)

# 空の要素や空白だけの要素を除外し、前後の空白をトリム
documents = [chapter.strip() for chapter in chapters_raw if chapter.strip()]

print("ドキュメントサイズ: ", len(documents))
# リストが空でないことを確認してからアクセスする
if documents: # もし documents リストが空でなければ
    print("\n最初のドキュメントの例: \n", documents[0])
    if len(documents) > 1: # 2つ以上のドキュメントがあれば2つ目も表示
        print("\n2番目のドキュメントの例: \n", documents[1])
    # すべてのドキュメントを順番に表示することもできる
    print("\n--- 全てのドキュメント ---")
    for i, doc in enumerate(documents):
        print(f"\n--- ドキュメント {i+1} (インデックス {i}) ---")
        print('\n'.join(doc.splitlines()[:10])) # 各ドキュメントの最初の10行を表示
else:
    print("ドキュメントが見つかりませんでした。raw_writedown の内容や split_pattern を確認してください。")

ドキュメントサイズ:  15

最初のドキュメントの例: 
 #

2番目のドキュメントの例: 
 第1章 総 則

# \(目 的\)

# 神戸情報大学院大学学則

第1条<br>神戸情報大学院大学\(以下「本大学院」という。\)は、人間力を有する高度 ICT<br>人材の育成を目的とする。

\(自己評価\)

第2条<br>2<br>本大学院は、前条の目的を達成するため、教育研究活動の点検及び評価を行う。<br>前項の点検及び評価の組織及び方法は、これを別に定める。

\(課 程\)

第3条<br>本大学院に設置する課程は、専門職学位課程とする。

#

--- 全てのドキュメント ---

--- ドキュメント 1 (インデックス 0) ---
#

--- ドキュメント 2 (インデックス 1) ---
第1章 総 則

# \(目 的\)

# 神戸情報大学院大学学則

第1条<br>神戸情報大学院大学\(以下「本大学院」という。\)は、人間力を有する高度 ICT<br>人材の育成を目的とする。

\(自己評価\)


--- ドキュメント 3 (インデックス 2) ---
第2章 研究科、修業年限及び収容定員

# \(研究科\)

第4条<br>本大学院に、情報技術研究科を置く。

\(課程·専攻\)

第5条<br>本大学院に、次の課程、専攻を置く。


--- ドキュメント 4 (インデックス 3) ---
第3章 学年、学期及び休業日

# \(学年及び学期\)

第8条<br>本大学院の学年は、毎年4月1日に始まり、翌年3月31日に終わる。

2<br>学年は、次の6期に分ける。

\(休業日\)


--- ドキュメント 5 (インデックス 4) ---
第4章 授業科目及び履修方法

# \(授業科目及び単位数\)

第10条 本大学院における授業科目及び単位数は、神戸情報大学院大学履修規程の定める通<br>りとする。

\(単位計算方法\)

第11条 各授業科目の単位数は、1単位の履修時間を教室及び教室外を合わせて45時間と<br>し、次の基準により計算するものとする。


--- ドキュメント 6 (インデックス 5) ---
第5章 課程修了の認定等

2


\(課程修了の認定等\)

第15条 課程修了の認定は、第6条に定

In [45]:
# Retrievalの実行
question =  "神戸情報大学院大学学則において定められている修業年限の限度は？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[55.951683044433594, 76.59706115722656, 77.3612060546875, 70.0725326538086, 73.6505126953125, 68.80538177490234, 73.37699127197266, 68.74102020263672, 66.43501281738281, 68.67029571533203, 62.57657241821289, 63.81586837768555, 66.9779281616211, 60.87345504760742, 71.12594604492188]]


In [46]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 77.3612060546875)
第2章 研究科、修業年限及び収容定員

# \(研究科\)

第4条<br>本大学院に、情報技術研究科を置く。

\(課程·専攻\)

第5条<br>本大学院に、次の課程、専攻を置く。

情報技術研究科

専門職学位課程 情報システム専攻

\(修業年限及び在学年限\)

第6条<br>本大学院の修業年限は、2年を標準とする。

但し、主として実務の経験を有する者に対して教育を行う場合あるいは教育上の必<br>要があると認められる場合、かつ教育上支障を生じないと認められる場合におい<br>て、教授会での審議を経て、学長の承認のもとに一年以上二年未満の期間を修業年<br>限とすることができる。

2<br>本大学院の在学期間は、4年間を限度とする。

\(収容定員\)

第7条<br>本大学院の収容定員は、次の通りとする。

情報技術研究科

情報システム専攻

入学定員 55名<br>収容定員 110名 


取得したドキュメント2: (Score: 76.59706115722656)
第1章 総 則

# \(目 的\)

# 神戸情報大学院大学学則

第1条<br>神戸情報大学院大学\(以下「本大学院」という。\)は、人間力を有する高度 ICT<br>人材の育成を目的とする。

\(自己評価\)

第2条<br>2<br>本大学院は、前条の目的を達成するため、教育研究活動の点検及び評価を行う。<br>前項の点検及び評価の組織及び方法は、これを別に定める。

\(課 程\)

第3条<br>本大学院に設置する課程は、専門職学位課程とする。

# 


取得したドキュメント3: (Score: 73.6505126953125)
第4章 授業科目及び履修方法

# \(授業科目及び単位数\)

第10条 本大学院における授業科目及び単位数は、神戸情報大学院大学履修規程の定める通<br>りとする。

\(単位計算方法\)

第11条 各授業科目の単位数は、1単位の履修時間を教室及び教室外を合わせて45時間と<br>し、次の基準により計算するものとする。

\(1\) 講義、演習については、教室内における1時間の講義、演習に対して教室外<br>における2時間の準備のための学習を必要とするもの

In [47]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 神戸情報大学院大学学則において定められている修業年限の限度は？"
response_off = generate_output_qwen3(query, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF with RAG: {response_off}")

Thinking OFF with RAG: 神戸情報大学院大学学則において定められている**修業年限の限度**は、**4年間**です。

### 解説：
- **第6条**において、修業年限は**2年を標準**とし、**4年間を限度**とされています。
- ただし、実務経験者や教育上必要な場合に、**1年以上2年未満の期間**で修業年限を設定することも認められています。
- したがって、**最長の修業年限は4年間**です。


In [ ]:
question =  "神戸情報大学院大学はどこにある？"
response_on = generate_output_qwen3(question, model, tokenizer, enable_thinking=True)
print(f"Thinking ON: {response_on}")
response_off = generate_output_qwen3(question, model, tokenizer, enable_thinking=False)
print(f"Thinking OFF: {response_off}")

In [34]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

0.0


### 結果 (初期RAG実装)

講義内容のドキュメントを追加したにもかかわらず、モデルの回答には依然として以下の問題が見られます：
* 「高速に推論する」など、従来の一般的な推論最適化と「Inference Time Scaling」を混同した誤った解釈が継続
* 講義内容を参照しているものの、概念の本質を正確に捉えられていない

### 問題分析
以下の要因が考えられます：
1. **ドキュメント品質の問題**: 音声認識による文字起こしの精度不足
2. **検索精度の課題**: 単純な文単位の分割では文脈が失われ、関連性の高いドキュメント片を適切に取得できていない可能性

## 2.2 書き起こしテキストの品質改善

日本語の音声認識（speech2text）モデルは精度に課題があることが知られています。以下に「LLMにおけるInference Time Scalingとは？」に関連する講義内容の書き起こしテキストを比較します：

### 講義中の該当発言 (LLM講座Day4後半から抜粋)
---

<修正前>
---

講義に戻ります。ちょっと練習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドって話で**生のGENIACLM**の話をして終わろうと思いですねちょっとモチベーションから話すと、ちょっと頭で考えてみてほしいとか見れば一瞬で思うとんですけどバナナの色は何ですかって言われたときと、今日の講義聞いた上で、**ゲームソフトの問題は何だと思いますか**って聞かれたとき、多分あの考えることが違うと思うんですね。**羽の色なんですか**っていうと一瞬黄色ですねもしかしたら緑かもしれないけどぐらいですかね物によるかなみたいなおもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**スケール足の問題なんだと思いますか**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだからどうだろうこの辺が問題かなみたいな考えとやっぱ思考としては違うってことは何となく思うかなと思います。なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**スケールって言ってる7Gのスケール**って言ってるのはこういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、あの考えなきゃいけない問題に対しては、考える時間を、に計算式を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。二つで、ちょっと順番が前後しますけどこれの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**大湾**と呼ばれる論文が論文じゃないか、モデルが**オペル**から出ましたプレビューとして出てますけどこの法案で注目されていますこれあの**論文にROMってかブログ**にあるとイエスって右側が訓練時の計算資源をスケールさせたときに、初めて何かロジックのベンチマークがあるんですけどこれをがどうなったかで何となくスケールしてると右側がテストTimeコンピュートっていうふうに書いてると思うんすけど、**水温時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだんコース計算式を増やしていったときに、性能がどう変わるかっていうのでこれもスケールしていってるということがわかると思います。こういった形で、要は考える時間をどうやら推論時に使うと計算資源を推論使うのはいいことがありそうだということがわかります。

---
<修正後>
---

講義に戻ります。ちょっと演習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドってことで**「推論時のスケーリング」**についての話をして終わろうと思います。モチベーションから話すと、ちょっと頭で考えてみてもらえれば一瞬でわかると思うとんですけど、「バナナの色は何ですかって言われたとき」と、今日の講義聞いた上で、**「スケール則の問題は何だと思いますか」**って聞かれたとき、多分あの考えることが違うと思うんですね。
**「バナナの色なんですか」**っていうと黄色ですね。もしかしたら緑かもしれないけど、物によるかなみたいな、おもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**「スケール則の問題なんだと思いますか」**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだから「どうだろう」「この辺が問題かな」みたいな考えとはやっぱ思考としては違うってことは何となく思うかなと思います。
なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**推論時のスケールって言ってるのは**こういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、深く考えなきゃいけない問題に対しては、考える時間に計算資源を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。
これの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**o1**と呼ばれるモデルが**OpenAI**から出ました。プレビューとして出てますけどこのo1で注目されています。これあのo1の**論文ってかブログ**にある図で、左側が訓練時の計算資源をスケールさせたときに、AIMEというロジックのベンチマークがあるんですけど、accuracyがどうなったかというと、何となくスケールしてる。右側がtest-time computeっていうふうに書いてると思うんすけど、**推論時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだん計算資源を増やしていったときに、性能がどう変わるかっていうので、これもスケールしていってるということがわかると思います。
こういった形で、要は考える時間をどうやら推論時に使うと、つまり計算資源を推論時に使うのはいいことがありそうだということがわかります。






---
### 文字起こしの誤り

上記の比較からわかるように、音声認識による書き起こしには重大な誤りが多数含まれています：
* 「スケール則の問題」→「ゲームソフトの問題」
* 「o1」→「大湾」
といった明らかに文脈に合わない単語変換が発生しています。

`LLM2024_day4_raw.txt`の中には、このような誤変換が多数見られます。これらの誤りはRAG性能に直接影響し、モデルの回答精度を低下させる要因となります。

したがって、**ドキュメント品質の改善**を行い、RAG性能の向上を図ります。

## 再開

## 講義内容をソースとして活用：改善版RAG実装

* **ドキュメント処理**:
  - speech2textによる書き起こしテキストを人手で丁寧に修正
  - 専門用語（Inference Time Scaling、GPT-o1など）の正確な表記を確保
  - 文脈の流れを維持しつつ、文法的に正確な日本語に修正

* **検索手法**:
  - 引き続き「。」（句点）で区切られた文単位でテキストを分割
  - 文単位の検索により、モデルの入力トークン制限内で関連情報を最大化

この改善により、モデルが正確な情報に基づいて「Inference Time Scaling」の概念を理解し、適切な回答を生成することが期待されます。

In [35]:
with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4.txt", "r") as f:
  raw_writedown = f.read()

In [36]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[310])

ドキュメントサイズ:  350
ドキュメントの例: 
 それからBest of Nとはちょっと違う方法として、N個を生成した後に、それらを集約するという意味では、Day2でやったSelf-Consistencyをこの枠組みの一つとして説明されます


In [37]:
# Retrievalの実行
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[61.82585906982422, 66.46412658691406, 60.23835372924805, 51.8336296081543, 54.9107551574707, 51.56048583984375, 58.39980697631836, 60.80604934692383, 55.0720100402832, 59.28540802001953, 58.47719955444336, 62.86042785644531, 58.22374725341797, 59.10657501220703, 55.343997955322266, 57.3587646484375, 59.760581970214844, 51.49699020385742, 60.624359130859375, 62.524269104003906, 57.502044677734375, 58.56466293334961, 59.720706939697266, 59.44036102294922, 63.487945556640625, 61.78038024902344, 59.621238708496094, 62.45943832397461, 53.73781967163086, 61.90666961669922, 55.26149368286133, 56.99132537841797, 62.45279312133789, 56.23671340942383, 56.53040313720703, 56.40115737915039, 58.694862365722656, 51.75257873535156, 56.227569580078125, 56.70087432861328, 58.3699951171875, 55.44434356689453, 59.27926254272461, 59.53657531738281, 57.6950798034668, 54.768287658691406, 58.02783966064453, 57.402347564697266, 56.4503059387207, 52.4367561340332, 61.598182678222656, 62.70309066772461, 57.00

In [38]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 67.38106536865234)
最後に補足して僕のパート終わろうと思いますけど、同じ計算資源のときにパラメータ増やすのよりも推論資源を増やすのが有効なのかっていうのが問いとしてあると思いますけど、o1の場合だと、訓練時のスケールは同じままって推論時のスケールを増やしたら、より賢くなりましたって話でしたけど、どっちにするのがいいのかっていう意味で言うと、GoogleDeepMindが8月に論文としてまして、Scaling LLM Test-Time Comupte Optimally can be more Effective than Scaling More Paremetersっていうことで、良いらしいというふうに言われてます 


取得したドキュメント2: (Score: 66.58795166015625)
右側がtest-time computeっていうふうに書いてると思うんすけど、推論時に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだん計算資源を増やしていったときに、性能がどう変わるかっていうので、これもスケールしていってるということがわかると思います 


取得したドキュメント3: (Score: 66.46412658691406)
あのスケールするっていうところではタイトルの通りなんですけど、ちょっとこれスケーリングPretraining回ってなってるんですけれども、ちょっと最近はですね、このPretrainingだけではなくて、推論をスケールさせるというような話も出てきてましてせっかくなのでその最近の話題ということです推論時のスケーリングことで、ちょっとタイトル詐欺が入ってるんですけどPretrainingだけじゃない、スケーリングも扱うということで、ちょっと若干あのタイトル詐欺なんですけども、あの最近の話題ということで推論時のスケジュールについても話していきたいなと思っています 


取得したドキュメント4: (Score: 66.20881652832031)
Trasnformerの場合はスケール則が、パラメータ数が横軸になってますけどこういうふうになると、LSTMの場合には1層2層4層みたいにそれぞれスケール則を解くとこんなふうになります

In [39]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
response = generate_output(query)
print(response)

## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、**推論時に計算資源を増やす**ことを指します。 

**具体的には:**

* **Pretraining**段階でモデルのパラメータ数を増やす
* **推論時**に、計算資源を増やすことで、モデルの性能を向上させる

**GoogleDeepMindの研究**では、**推論時のスケール**を**最適化**することで、**パラメータ数の増やすよりも、推論資源を増やす方が効果的**であることが示唆されています。


**スケールする際の具体的な方法:**

* **Pretraining**: モデルの学習段階で、パラメータ数を増やす
* **推論時**:  計算資源を増やして、モデルの処理能力を向上させる


 





In [40]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

2.0


## 結果 (修正テキストによるRAG)

書き起こしテキストの品質改善により、モデルの回答に部分的な向上が見られました：

### 改善点
* 「推論時に計算資源を増やすこと」という概念を正確に捉えるようになった

### 問題点
* 「Pretraining時」という記述は講義内容と矛盾している

### 問題分析

モデルが誤った回答を生成する主要因として、**文脈の欠如**が考えられます：
* 「。」で区切られた短い文単位での検索では、各文の発言背景や関連性が失われる
* 単独の文から情報を抽出するため、講師の全体的な主張や議論の流れを把握できない
* 結果として、正しい個別の文でも、その解釈に必要な背景情報が欠如し、誤った文脈で理解される


---



# 3. 文脈を考慮したチャンク化の導入

検索結果の品質向上のため、以下の改善を実施します：

* **前後文脈を含むチャンク化**:
  - 検索でマッチした文だけでなく、その前後の複数文も含めてチャンクとして取得
  - 具体的には、マッチした文を中心に前2文、後2文を含む計5文程度のチャンクを構成
  - この「文脈ウィンドウ」により、発言の背景情報や議論の流れが保持される

* **期待される効果**:
  - 講師の主張とその根拠の関係性を正確に把握できる
  - 概念の定義とその適用範囲を正しく理解できる

この改善により、モデルが講義内容の本質をより正確に理解し、一貫性のある事実に基づいた回答を生成することが期待されます。

In [41]:
# 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
response = generate_output(query)
print(response)

## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、**推論時の計算資源を増やすことで、モデルの性能を向上させる**ことを指します。 

具体的には、**モデルの学習時に使用する計算資源と、推論時に使用する計算資源のバランス**を調整することで、より高速で正確な推論を実現します。


**Inference Time Scalingのポイント**

* **スケーリングの目的:**  推論時の計算資源を増やすことで、モデルの性能を向上させる
* **スケーリング方法:**  Pretraining段階でのスケーリングに加え、推論段階でのスケーリングも検討
* **影響:**  スケーリングによって、モデルの精度や処理速度が変化する
 



 





In [42]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

2.0


## 結果 (文脈付きチャンク化によるRAG)

文脈を含むチャンク化により、モデルの回答の方向性に明確な改善が見られました：

### 改善点
* 「推論時の計算をスケールさせる」という概念を据えて回答
* Inference Time Scalingの基本原理についての理解が向上

### 残存する問題点
* 質問と関連性の低い情報（ノイズ）が混入する

### 問題分析

文脈付きチャンク化によるアプローチで新たに発生した課題：

1. **情報過多の問題**:
   * ドキュメント量の増加により、モデルに提供される情報総量が大幅に増加
   * 関連情報と非関連情報が混在し、ノイズと重要情報の区別が困難に

2. **情報選択の複雑化**:
   * モデルは単に回答を生成するだけでなく、提供された多様な情報源から関連性の高い情報を選別する作業も担うことになった
   * この二重タスクにより回答生成の難易度が上昇




# 4. Rerankによる情報品質の向上

検索精度をさらに向上させるため、二段階の検索プロセスを導入します：

* **Rerank手法の導入**:
  - 第一段階: 従来通り基本的な検索アルゴリズムでtop-k個のドキュメントチャンクを取得
  - 第二段階: 取得したチャンクに対してLLMを活用した高度な関連性評価を実施
  - LLMに「このドキュメントは質問『LLMにおけるInference Time Scalingとは？』に対して本当に関連性が高いか」を判断させる
  - 関連性スコアに基づいてランク付けし、真に関連性の高いチャンクのみを選出

* **期待される効果**:
  - 質の高い情報に焦点を絞ることで、ノイズとなる情報を大幅に削減
  - 文脈を保ちながらも、関連性の高い情報のみをモデルに提供
  - モデルのタスクを「多量の情報から選別して回答」から「厳選された情報に基づいて回答」へと単純化

この高度な情報フィルタリングにより、Inference Time Scalingに関する正確で一貫性のある回答生成が期待されます。

In [43]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = []
for ref in ["。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:

  query = f"与えられた[参考資料]が[質問]に直接関連しているかを、'yes''no'で答えること。[参考資料]\n{ref}\n\n[質問] LLMにおけるInference Time Scalingとは？"
  response = generate_output(query)

  print("\n\n対象となるドキュメント:\n", ref.replace("。", "。\n"))
  print("\n関連しているかどうか: ", response)

  if "yes" in response.lower():
    references.append(ref)



対象となるドキュメント:
 これでほぼちょうどですけど、最後に少しあの、前半では全体の訓練時のスケーリングをする話を基本的にしましたけど、最近ではこの推論時の計算量っていうのも注目するような研究が増えてきています。
代表的なGPT-o1とかですごく注目されてるかなと思いますし、今までやった方法、学んだ方法も結構出てきたと思いますけど、Promptingを工夫するとか、Decodingを工夫するとかいうので、それにも発展的な方法がいろいろ出てきていますし、Meta Generationっていうような枠組みで、DecodingだけじゃなくてそのDecodeした結果を最後どう使うかみたいな含めて、Meta Generationというふうに呼んでますけど、Paralell SearchとかStep Level SearchとかRefinementと言われるような枠組みの研究も出てきていますというような話をしました。
最後に補足して僕のパート終わろうと思いますけど、同じ計算資源のときにパラメータ増やすのよりも推論資源を増やすのが有効なのかっていうのが問いとしてあると思いますけど、o1の場合だと、訓練時のスケールは同じままって推論時のスケールを増やしたら、より賢くなりましたって話でしたけど、どっちにするのがいいのかっていう意味で言うと、GoogleDeepMindが8月に論文としてまして、Scaling LLM Test-Time Comupte Optimally can be more Effective than Scaling More Paremetersっていうことで、良いらしいというふうに言われてます。
厳密に言うとこれなんかタスクによって違うということなので、良いとまで言っていいのかちょっと若干誇大広告な気が個人的にはしてますけど、そういったことを検証するような研究も出てきていますので興味ある人は見てもらえばと思います

関連しているかどうか:  yes 





対象となるドキュメント:
 プレビューとして出てますけどこのo1で注目されています。
これあのo1の論文ってかブログにある図で、左側が訓練時の計算資源をスケールさせたときに、AIMEというロジックのベンチマークがあるんですけど、accuracyがどうなったかというと、何となくスケールしてる。
右

In [44]:
print(len(references))

4


上記より、上位4件のみが関連しているとわかったので、これらだけをモデルに渡すこととする。（生成内容が確立的なので、4件でない可能性もあります）

In [45]:
query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
response = generate_output(query)
print(response)

## LLMにおけるInference Time Scalingとは？

**Inference Time Scaling**とは、**推論時の計算資源を増やす**ことで、**モデルの性能を向上させる**ことを指します。 

具体的には、**訓練時のスケール**と**推論時のスケール**の両方を検討し、**最適なスケール設定**を見つける研究が進んでいます。


**Inference Time Scaling のポイント**

* **スケールする対象**:  従来は**Pretraining**のみを対象としていましたが、近年では**推論時のスケール**も重要視されています。
* **目的**:  **推論時の計算資源を増やす**ことで、**モデルの性能を向上**させることを目指しています。
* **研究**:  **Scaling Law**と呼ばれる研究で、**Pretraining**だけでは**性能に影響がない**という結果も出ています。
* **スケール方法**:  **モデルのDimension、埋め込みの数次元**などを考慮し、**最適なスケール設定**を見つけることが重要です。




 





In [46]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

2.0


## 結果 (Rerank導入後)

Rerankの導入により、回答品質に改善が見られました：

### 達成された成果
* Inference Time Scalingに関する正確な情報を含んだ回答の生成
* 無関係な情報やノイズの排除
* 講義内容を反映した説明の実現 🎉

この結果から、RAGパイプラインにおける情報の質と関連性の重要性であり、検索で取得した情報を単に増やすだけでなく、その情報の関連性を精査する方法を学ぶことができました。

# 5. さらなる改善案: 意味的チャンク化

文単位での分割と前後文脈の追加という現在のアプローチをさらに発展させる手法として、**意味的なチャンク化**が考えられます：

* **意味的チャンク（段落）単位での分割**:
  - 単純な文の区切りではなく、意味的なまとまり（トピック、議論、例示など）に基づいてテキストを分割
  - 人間の主観に基づく意味的な段落分けを活用
  - 各チャンクが「一つの完結した考え」を表現するようにする

* **期待される効果**:
  - より自然な文脈理解が可能に（人間の思考や会話の流れに近い）
  - トピックの開始から結論までの流れを維持できる
  - 概念間の関係性や比較が同一チャンク内に含まれ、より深い理解につなげる

* **検証方法**:
  - 人間が主観的に意味でグループ化したチャンクセットを用意
  - 同じRerank手法を適用し、文単位チャンクとの性能差を比較
  - 回答の正確性、一貫性、網羅性を評価指標として使用

この意味的チャンク化手法は、特に講義のような構造化された発話においては、より自然で効果的な情報検索と理解を可能にすると予想されます。

**注意事項**

**ここから先のセルを実行した場合、GPUメモリ不足になる可能性が高いです。**


このノートブックでは、GPUはT4を使用しています。
Colab Pro等を契約し、L4などのよりGPUメモリの大きいものを使用するか、モデルやその設定等を変更するなどの工夫が必要になります。

In [47]:
# 本来は段落をそのままdocumentsに入れずに一定のサイズに分割した方が良いでしょうが、簡単のために段落をそのまま入れてしまいます。
documents = [text.replace("\n", " ").strip() for text in raw_writedown.split("\n\n")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[30])

ドキュメントサイズ:  45
ドキュメントの例: 
 具体的な求め方についても話します。 さっきからチラチラ言ってた通りなんすけど基本的にこれどう図るかっていうと、基本的にはいくつかの条件で実験してフィッティングするって言ってんのは、すごい単純に言ってしまうとそうなります。左側GPT4の論文から取ってきた図で説明したもんですけど、グレーのやつを例えば実験してみて、これぐらいのロスになるんだなっていうので、フィッティングするとこういうカーブになります。 ちなみにこれ、なんでこれ直線にならないんだっていうのをすぐ説明しなかったですがこれ縦軸が実は普通のロスと違ってBits-per-wordっていうのになってて、多分2乗スケールのロスになってるからだと思います。 右側も同じですね。この各点について何かいろんな設定で実験してやって、それを結果を見るということをしてますけどよくよく考えるとスケールさせるときにモデルサイズどうすればいいんでしたっけとか、何をどういじるとモデルサイズが大きくなるんでしたっけ、どういうふうに言えばいいんでしたっけとかですね。 あのモデルサイズ変えたときにハイパーパラメータってどうすんでしたっけそういった細かい問題が出てくる。最初の方ですけどモデルサイズどう変化させるかっていうので、前回やった、こういう図があると思いますけどモデルサイズ変えようと思ったら別にパラメータ、層の数を増やしても、いいわけですし、この埋め込みの次元各tokenの次元を増やしてもいいわけですし、各随所に出てくるこのフィードフォワードネットワークっていうのの中間層の次元を上げてもいいですしヘッドを増やしてもそういうのあのパラメータ自体は上がるということで、これどれをどのぐらいやるんですかっていうのが細かく考えると重要になってきます。 この辺は元の論文でも一応議論されてまして、これ三つほど出してるんすけど例えば真ん中のがアスペクト比っていう、モデルのエンベディングのサイズですね。dモデルっていうものを層数で割ったもの、アスペクト比という縦横比みたいなもので幅と深さの比率をアスペクト比っていうふうにこの論文では呼んでいますけど。こういったものを変えて実験してみたっていうのが最初の最初じゃないOpenAIのScaling Lawで話されていました。基本的にはこの辺見るとなんかあんまり性

In [49]:
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

In [ ]:
# 簡単のためにtop2でやります。結果を見てもらえれば問題なく関連する項目のみ取得できているのが分かるかと思います。
topk = 2
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

In [ ]:
reference = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"
response = generate_output(query)
print(response)

In [ ]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)